## Pedicularis phylogeography project - GDM Data Prep
This is running locally on 00100 `/Users/isaac/miniconda3/envs/gdm/bin/jupyter-notebook --port=8890 --no-browser --log-level=50`

### Other notes
* [Jinsha River](https://maps.app.goo.gl/avHGiu44Dn9XQUKf7) is a major driver in community turnover according to some of Rick's previous work.
* [Yalong River](https://maps.app.goo.gl/cqBcDyiePBt1jmv1A) shows more genetic turnover in our Pedicularis phylogeography dataset

In [1]:
import elevation
import geopandas as gpd
import glob
import libpysal
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import rasterio
import re
import spopt
import subprocess
import sys

from rasterio.mask import mask as _mask
from rasterio.merge import merge
from rasterio.plot import show
from rpy2.robjects.packages import importr
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from shapely.geometry import Point

pd.set_option('display.max_columns', None)


## Peducularis data from the pdf supplement of Liu et al 2024

Names to fix:
* Pedicularisobliquigaleata
* Pedicularis_sp.2_(jiaozishansis)
* Pedicularis_aff._amplituba
* Pedicularis_aff._sigmoidea

In [2]:
# Peducularis data from the pdf supplement of Liu et al 2024
df = pd.read_csv("../data/Liu_et_al_2024-Cladistics.txt", sep=" ")
df["geometry"] = df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
display(df.head(2))

# Convert to a GeoDataFrame
liu_gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")  # WGS 84 (lat/lon)

liu_gdf.explore("Name", legend=False, tiles="CartoDB positron")


,Name,Longitude,Latitude,geometry
0,Pedicularis_siphonantha,85.98706,28.03192,POINT (85.98706 28.03192)
1,Pedicularis_siphonantha,94.67609,29.61145,POINT (94.67609 29.61145)


## Pedicularis data from Eaton Lab

The Field Data for all Eaton Lab Pedicularis samples lives in a google sheet - [Field notes: 2018-2022](https://docs.google.com/spreadsheets/d/1s2WTV5PSMozhFE-VXS2ekwUQIVcsdw4Zu6ABL04EyBM/edit?gid=1599831623#gid=1599831623).

**Fix these two weird ones by hand in the csv:**
* KY03 - rhynchodonta, rather than cheilanthifolia
* KY05 - brevialbris?


In [4]:
def dms_to_dd(dms):
    # Input is Lat/Long in this format: D˚M'S" N
    # Output is decimal degrees
    try:
        # If it's already decimal degrees cast to float
        dd = float(dms)
    except:
        # It turns out there are 2 different 'degrees' characters that are mixed in the field notes file
        # '°' == '˚' <- False, so replace all instances of one with the other
        dms = dms.replace('°', '˚')
        #Parse the dms format and cast to floats. Ignore the direction because they are all N/E
        degrees, minutes, seconds = [float(x) for x in re.split(r"[˚'\" ]+", dms)[:3]]
        dd = degrees + (minutes / 60) + (seconds / 3600)
    return dd

In [5]:
dae_df = pd.read_csv("../data/eaton_lab_fieldnotes.csv")
# Convert degrees in dms format to decimal
dae_df['latitude'] = dae_df['latitude'].apply(dms_to_dd)
dae_df['longitude'] = dae_df['longitude'].apply(dms_to_dd)

# Remove any subspecies or variety information from the species epithet
dae_df["species_epithet"] = dae_df["species_epithet"].str.split().str[0]

# Join the genus/species columns to agree with the Liu data
dae_df["Name"] = dae_df["genus"] + "_" + dae_df["species_epithet"]
# Retain only the name and lat/long
dae_df = dae_df[["Name", "longitude", "latitude"]]
# Rename columns for agreement
dae_df.rename(columns={"longitude":"Longitude", 
                    "latitude":"Latitude"}, inplace=True)

# Cleaning NA
pre = len(dae_df)
dae_df = dae_df.dropna()
print(f"Removed nan samples: {pre - len(dae_df)}")

# Removing AK samples
mask = dae_df['Latitude'] < 60
pre = len(dae_df)
dae_df = dae_df[mask]
print(f"Removed AK samples: {pre - len(dae_df)}")

# Selecting only Pedicularis
mask = dae_df["Name"].str.contains("edicularis")
#display(dae_df[~mask])
pre = len(dae_df)
dae_df = dae_df[mask]
print(f"Removed non-Pedicularis samples: {pre - len(dae_df)}")

# Remove unidentified sp. samples
mask = dae_df["Name"].str.endswith(('sp', 'sp.', 'spp.'))
dae_df = dae_df[~mask]
print(f"Removed samples w/o species id: {sum(mask)}")

print(f"Total retained samples: {len(dae_df)}")

# Set geometry for sample points
dae_df["geometry"] = dae_df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)

# Convert to a GeoDataFrame
dae_gdf = gpd.GeoDataFrame(dae_df, geometry="geometry", crs="EPSG:4326")  # WGS 84 (lat/lon)
dae_gdf.explore("Name", legend=False, tiles="CartoDB positron")

Removed nan samples: 2
Removed AK samples: 22
Removed non-Pedicularis samples: 45
Removed samples w/o species id: 15
Total retained samples: 912


## Check names

In [6]:
liu_names = set(liu_gdf["Name"])
dae_names = set(dae_gdf["Name"])
print(sorted(liu_names))
print(sorted(dae_names))

['Pedicularis_aff._amplituba', 'Pedicularis_aff._sigmoidea', 'Pedicularis_amplituba', 'Pedicularis_armata', 'Pedicularis_aschistorrhyncha', 'Pedicularis_axillaris', 'Pedicularis_cephalantha', 'Pedicularis_chinensis', 'Pedicularis_corymbifera', 'Pedicularis_cranolopha', 'Pedicularis_crenata', 'Pedicularis_crenularis', 'Pedicularis_davidii', 'Pedicularis_decorissima', 'Pedicularis_delavayi', 'Pedicularis_dichrocephala', 'Pedicularis_dissectifolia', 'Pedicularis_dolichantha', 'Pedicularis_dolichosiphon', 'Pedicularis_dulongensis', 'Pedicularis_elwesii', 'Pedicularis_fastigiata', 'Pedicularis_fengii', 'Pedicularis_fletcheri', 'Pedicularis_franchetiana', 'Pedicularis_gagnepainiana', 'Pedicularis_gruina', 'Pedicularis_gyirongensis', 'Pedicularis_henryi', 'Pedicularis_hongii', 'Pedicularis_humilis', 'Pedicularis_kariensis', 'Pedicularis_labordei', 'Pedicularis_lanpingensis', 'Pedicularis_latituba', 'Pedicularis_leptosiphon', 'Pedicularis_limprichtiana', 'Pedicularis_longicalyx', 'Pedicularis_

## Merge all datasets
Remove samples west of 93 deg long (far West of any of our samples).

In [325]:
long_cutoff = 93

full_gdf = pd.concat([liu_gdf, dae_gdf])
len_full = len(full_gdf)
print(f"Len full dataset - {len_full}")
full_gdf = full_gdf[full_gdf["Longitude"] > long_cutoff]
print(f"Removed # samples W of {long_cutoff} - {len_full - len(full_gdf)}")

m = full_gdf.explore("Name", legend=False, tiles="CartoDB positron")
m

Len full dataset - 1656
Removed # samples W of 93 - 40


## Convex hull around full dataset

In [326]:
# Convex hull buffered by 100km
hull = full_gdf.to_crs(epsg=3857).dissolve().convex_hull.buffer(100000)
hull = hull.to_crs(epsg=4236)

hull.explore(m=m, style_kwds={"fillOpacity":0.15, "opacity":0.25})

## Fetch & merge DEM tiles & Bioclim data

The data will end up living here:
../env_data/SRTM_DEM.tif
../env_data/wc10m/*

**This only needs to be done once.**

But it needs to be redone if you add samples that fall outside the tiles already downloaded.

#### Fetch the bioclim data
Get it from here: https://www.worldclim.org/data/worldclim21.html

* 10' resolution for testing
* 30" resolution for reality

In [110]:
sys.exit("Comment this line to actually run this again")

# Step 2: Get unique tile coordinates (1x1 degree SRTM tiles)
gdf['lat_tile'] = np.floor(gdf['Latitude']).astype(int)
gdf['lon_tile'] = np.floor(gdf['Longitude']).astype(int)
tiles = gdf[['lat_tile', 'lon_tile']].drop_duplicates()

print(f"Found {len(tiles)} unique SRTM tiles to download.")

# Step 2: Clip each tile to its 1x1° bounding box and save
tile_paths = []
for i, row in tiles.iterrows():
    lat, lon = row['lat_tile'], row['lon_tile']
    bounds = (lon, lat, lon + 1, lat + 1)
    out_file = f"tile_{lat}_{lon}.tif"
    
    print(f"Clipping SRTM tile for bounds: {bounds}")
    elevation.clip(bounds=bounds, output=out_file)
    tile_paths.append(out_file)

# Merge DEM tiles
# `elevation` package saves cache files to `~/Library/Caches/elevation/SRTM1`

# Step 1: Collect all the clipped tiles (e.g., from earlier `elevation.clip()` steps)
tile_cache_dir = "/Users/isaac/Library/Caches/elevation/SRTM1"
tile_files = glob.glob(f"{tile_cache_dir}/tile_*.tif")  # or a custom list if needed

# Step 2: Open all tiles
src_files_to_mosaic = [rasterio.open(fp) for fp in tile_files]

# Step 3: Merge
mosaic, out_transform = merge(src_files_to_mosaic)

# Step 4: Copy metadata
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_transform
})

# Step 5: Write to disk
with rasterio.open("SRTM_DEM.tif", "w", **out_meta) as dest:
    dest.write(mosaic)

SystemExit: Uncomment this line to actually run this again

/Users/isaac/miniconda3/envs/gdm/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Clip the DEM and Bioclim variable to the convex hull

**This only needs to be done once.**

But it needs to be redone if you add samples that fall outside the tiles already downloaded.

Clipped raster data will end up living here:
../env_data/hengduan_dem.tif
../env_data/hengduan_bc10m/*

In [161]:
%%time

sys.exit("Comment this line to actually run this again")

# Elevation
print("Processing DEM")
with rasterio.open('../env_data/SRTM_DEM.tif') as src:
    # Reproject points to DEM CRS if needed
    if hull.crs != src.crs:
        hull = hull.to_crs(src.crs)

    # Clip the raster using the convex hull polygon geometry
    out_image, out_transform = _mask(src, hull.geometry, crop=True)
    out_meta = src.meta.copy()

# Update metadata for the clipped raster
out_meta.update({
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Save the clipped raster
with rasterio.open("../env_data/hengduan_dem.tif", "w", **out_meta) as dest:
    dest.write(out_image)

# -------------------------
# Bioclim
# -------------------------
try:
    clipped_raster_dir = "../env_data/hengduan_wc30s"
    os.mkdir(clipped_raster_dir)
except FileExistsError:
    pass

bioclims = glob.glob("../env_data/wc30s/*.tif")
for clim in bioclims:
    # Get the bioclim variable name
    var = clim.split('.')[-2].split("_", 2)[-1]

    print(f"Processing: {var}")
    with rasterio.open(clim) as src:
        # Reproject points to DEM CRS if needed
        if hull.crs != src.crs:
            hull = hull.to_crs(src.crs)

        # Clip the raster using the convex hull polygon geometry
        out_image, out_transform = _mask(src, hull.geometry, crop=True)
        out_meta = src.meta.copy()
    
    # Update metadata for the clipped raster
    out_meta.update({
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    # Save the clipped raster
    with rasterio.open(f"{clipped_raster_dir}/{var}.tif", "w", **out_meta) as dest:
        dest.write(out_image)


Processing DEM
Processing: bio_16
Processing: bio_17
Processing: bio_15
Processing: bio_14
Processing: bio_10
Processing: bio_11
Processing: bio_13
Processing: bio_12
Processing: bio_7
Processing: bio_6
Processing: bio_4
Processing: bio_5
Processing: bio_1
Processing: bio_2
Processing: bio_3
Processing: bio_8
Processing: bio_9
Processing: bio_19
Processing: bio_18
CPU times: user 37.4 s, sys: 39.9 s, total: 1min 17s
Wall time: 1min 30s


## Populate the gdf with elevation and bioclim information per site

Along the way we'll clip the rasters to the convex hull (+100km) around sampling sites

In [163]:
# Elevation
with rasterio.open('../env_data/hengduan_dem.tif') as src:
    # Reproject points to DEM CRS if needed
    if full_gdf.crs != src.crs:
        full_gdf = full_gdf.to_crs(src.crs)

    # Sample elevation at each point
    coords = [(point.x, point.y) for point in full_gdf.geometry]
    full_gdf['elevation'] = [np.int32(x[0]) for x in src.sample(coords)]

# -------------------------
# Bioclim
# -------------------------
bioclims = glob.glob("../env_data/wc10m/*.tif")
for clim in bioclims:
    with rasterio.open(clim) as src:
        # Reproject points to DEM CRS if needed
        if full_gdf.crs != src.crs:
            full_gdf = full_gdf.to_crs(src.crs)
    
        # Sample elevation at each point
        coords = [(point.x, point.y) for point in full_gdf.geometry]
        # Get the bioclim variable name
        var = clim.split('.')[-2].split("_", 2)[-1]
        full_gdf[var] = [x[0] for x in src.sample(coords)]

full_gdf.head()

,Name,Longitude,Latitude,geometry,elevation,bio_18,bio_19,bio_8,bio_9,bio_2,...,bio_7,bio_6,bio_11,bio_10,bio_12,bio_13,bio_17,bio_16,bio_14,bio_15
1,Pedicularis_siphonantha,94.67609,29.61145,POINT (94.67609 29.61145),4280,356.0,18.0,9.299000,-4.482917,10.830812,...,27.550499,-13.17575,-5.756792,9.299000,646.0,132.0,18.0,356.0,4.0,87.878700
4,Pedicularis_siphonantha,93.07282,28.62956,POINT (93.07282 28.62956),3525,245.0,12.0,8.783625,-6.090333,11.636374,...,28.241749,-14.62225,-6.090333,8.783625,391.0,95.0,12.0,245.0,3.0,100.388863
8,Pedicularis_siphonantha,93.35254,28.84324,POINT (93.35254 28.84324),4386,267.0,9.0,9.001041,-5.559333,12.568334,...,28.610001,-14.45000,-5.559333,9.001041,435.0,103.0,9.0,267.0,2.0,99.299995
9,Pedicularis_milliana,99.00950,28.38371,POINT (99.0095 28.38371),4357,330.0,34.0,9.324667,-3.807083,10.233854,...,25.898251,-12.24800,-5.078750,9.324667,664.0,132.0,31.0,330.0,7.0,72.334267
10,Pedicularis_milliana,99.01954,28.36896,POINT (99.01954 28.36896),4307,330.0,34.0,9.324667,-3.807083,10.233854,...,25.898251,-12.24800,-5.078750,9.324667,664.0,132.0,31.0,330.0,7.0,72.334267


## Prep data for clustering

GDM practitioners often rescale elevation when fitting models. Scale elevation so that a 1-meter 
elevational difference is equivalent to 100 meters of geographic distance — i.e., a ratio of 1:100.

In [327]:
def prep_data(gdf, include_elev=True, scale_elev=True, verbose=False):
    # This function does _NOT_ modify the passed in gdf
    #
    # include_elev: Toggle to include or not the elevation info for clustering
    # scale_elev: Whether to scale elevation to 'effective distance' in linear KM
    
    # convert to km based crs
    tmp_gdf = gdf.to_crs(epsg=3857)
    
    # Extract projected coordinates
    coords = np.array([[point.x, point.y] for point in tmp_gdf.geometry])
    
    # Scale coords to km
    coords = coords/1000
    
    if include_elev:
        elev_scaler = 1
        if scale_elev:
            # Scale elevation to 'effective distance in km', dividing by 1000m/km & multiplying
            # by 100 to scale 1:100 elevation/distance ratio (Ferrier et al.), so just divide by 10
            elev_scaler = 10
        coords = np.hstack([coords, tmp_gdf["elevation"].values.reshape(-1, 1)/elev_scaler])
        if verbose: print(coords[:5])
    return coords

coords = prep_data(full_gdf, include_elev=False)
coords

array([[10539.2941291 ,  3453.70250195],
       [10360.81892909,  3328.58128162],
       [10391.95721706,  3355.70908878],
       ...,
       [11339.30636859,  3505.2066976 ],
       [11339.30636859,  3505.2066976 ],
       [11339.30636859,  3505.2066976 ]], shape=(1616, 2))

# Create spatial clusters
GDM needs "communities" so we need to partition the individuals across the landscape into community as a function
of geographic and maybe environmental distance.

* [Spatial Clustering - Using AgglomerativeClustering](https://kazumatsuda.medium.com/spatial-clustering-fa2ea5d035a3)
* [Compare k-means, DBSCAN and Hierarchical Clustering](https://hex.tech/blog/comparing-density-based-methods/)
* [spopt - Spatial Optimization for python](https://pysal.org/spopt/)

## Clustering functions (dbscan/agglomerative)

Different clustering functions that will take in coords (and optionally elev) and return
a list of cluster membership IDs for each sample.

* DBSCAN - Can create really large contiguous blobs, and also assigns spatial outliers as -1 (lots of them)
* Agglom - Including scaled elevation can give weird results with clusters intermixed over large distances (presumably because of elev)

In [328]:
def clust_dbscan(coords, eps=75, min_samples=4):
    # Set DBSCAN parameters
    # eps: Distance threshold in km (adjust based on your data)
    # min_samples: Minimum points to form a cluster

    # Run DBSCAN clustering
    db = DBSCAN(eps=eps, min_samples=min_samples, metric="euclidean").fit(coords)

    print(f"Found {len(set(db.labels_))} clusters")
    return db.labels_

def clust_agglom(coords, distance_threshold=500, n_clusters=None):
    clustering = AgglomerativeClustering(distance_threshold=distance_threshold, n_clusters=n_clusters).fit(coords)
    print(f"Found {len(set(clustering.labels_))} clusters")
    return clustering.labels_

# Assign cluster labels to GeoDataFrame
full_gdf["cluster_agg"] = clust_agglom(coords, distance_threshold=100)
full_gdf["cluster_db"] = clust_dbscan(coords, eps=25)

Found 155 clusters
Found 65 clusters


In [340]:
clust_method = "cluster_agg"
full_gdf.explore(clust_method, tiles="CartoDB positron")

# Convert to projected coordinate system and save to file

In [342]:
tmp_gdf = full_gdf.to_crs(epsg=3857)
tmp_gdf.to_file("samples.geojson", driver="GeoJSON")
tmp_gdf.drop("geometry", axis=1).to_csv("samples.csv", index=False)

# Aggregate samples to cluster centroids

In [334]:
sites = full_gdf.to_crs(epsg=3857).dissolve(by=clust_method).centroid
site_latlongs = sites.to_crs(epsg="4326")
sites = gpd.GeoDataFrame(sites, columns=["geometry"])
sites.explore()

### Create the sites.csv file including bioclim data and latlongs per centroid site

In [336]:
# Bioclim
bioclims = glob.glob("../env_data/wc10m/*.tif")
for clim in bioclims:
    with rasterio.open(clim) as src:
        # Reproject points to DEM CRS if needed
        if sites.crs != src.crs:
            sites = sites.to_crs(src.crs)
        coords = [(point.x, point.y) for point in sites.geometry]
        # Get the bioclim variable name
        var = clim.split('.')[-2].split("_", 2)[-1]
        sites[var] = [x[0] for x in src.sample(coords)]
# Preserve lat/long as independent columns for the gdf
sites["Longitude"] = [point.x for point in sites["geometry"]]
sites["Latitude"] = [point.y for point in sites["geometry"]]

sites.to_crs(epsg=3857).to_file("sites.geojson", driver="GeoJSON")
sites.drop('geometry', axis=1).to_csv("sites.csv")
sites.head()

,geometry,bio_18,bio_19,bio_8,bio_9,bio_2,bio_3,bio_1,bio_4,bio_5,bio_7,bio_6,bio_11,bio_10,bio_12,bio_13,bio_17,bio_16,bio_14,bio_15,Longitude,Latitude
cluster_agg,,,,,,,,,,,,,,,,,,,,,,
0,POINT (94.67542 29.5346),356.0,18.0,9.299000,-4.482917,10.830812,39.312580,2.038177,617.491821,14.374750,27.550499,-13.175750,-5.756792,9.299000,646.0,132.0,18.0,356.0,4.0,87.878700,94.675420,29.534599
1,POINT (102.71725 31.92608),377.0,20.0,7.928250,-8.374042,12.425938,39.201015,0.090260,663.378113,14.333750,31.698000,-17.364250,-8.374042,7.928250,771.0,143.0,20.0,377.0,5.0,82.004707,102.717252,31.926083
2,POINT (100.24694 27.10044),385.0,52.0,13.375958,1.695708,11.187438,45.626698,7.511573,506.756439,17.722750,24.519501,-6.796750,0.989583,13.375958,805.0,146.0,46.0,385.0,11.0,67.397026,100.246939,27.100442
3,POINT (102.15136 34.07709),373.0,13.0,7.769000,-9.552167,13.985105,40.437786,-0.442594,704.003906,14.430500,34.584251,-20.153749,-9.552167,7.769000,696.0,135.0,13.0,373.0,2.0,89.052361,102.151361,34.077087
4,POINT (102.35762 30.16971),417.0,19.0,11.050792,-3.864417,10.844625,38.888790,3.995417,605.765747,16.179251,27.886250,-11.707000,-3.864417,11.050792,794.0,159.0,19.0,417.0,5.0,87.478752,102.357622,30.169715


### Create site x species matrix (including latlongs)

In [337]:
presence_absence = True
site_species_matrix = (
    full_gdf.groupby([clust_method, 'Name'])
       .size()
       .unstack(fill_value=0)  # fills missing combinations with 0
)
if presence_absence:
    site_species_matrix = (site_species_matrix > 0).astype(int)

site_species_matrix = pd.concat([site_species_matrix, sites[["Latitude", "Longitude"]]], axis=1)

# Rename the 'cluster_*' column to 'cluster' because to simplify transition to R/GDM
site_species_matrix.index.name = "cluster"
site_species_matrix.to_csv("site_species_matrix.csv")
site_species_matrix

,Pedicularis_aff._amplituba,Pedicularis_aff._sigmoidea,Pedicularis_alaschanica,Pedicularis_amplituba,Pedicularis_anas,Pedicularis_armata,Pedicularis_aschistorrhyncha,Pedicularis_axillaris,Pedicularis_batangensis,Pedicularis_bella,Pedicularis_bidentata,Pedicularis_binaria,Pedicularis_brachycrania,Pedicularis_brevilabris,Pedicularis_cephalantha,Pedicularis_cheilanthifolia,Pedicularis_chinensis,Pedicularis_chumbia,Pedicularis_cinerascens,Pedicularis_confertiflora,Pedicularis_corymbifera,Pedicularis_cranolopha,Pedicularis_crenata,Pedicularis_crenularis,Pedicularis_cristatella,Pedicularis_cryptantha,Pedicularis_cyathophylla,Pedicularis_cyathophylloides,Pedicularis_davidii,Pedicularis_debilis,Pedicularis_decora,Pedicularis_decorissima,Pedicularis_delavayi,Pedicularis_densispica,Pedicularis_dichotoma,Pedicularis_dichrocephala,Pedicularis_dissectifolia,Pedicularis_dolichantha,Pedicularis_dolichocymba,Pedicularis_dolichoglossa,Pedicularis_dolichosiphon,Pedicularis_dulongensis,Pedicularis_elwesii,Pedicularis_fastigiata,Pedicularis_fengii,Pedicularis_fetisowii,Pedicularis_fletcheri,Pedicularis_franchetiana,Pedicularis_gagnepainiana,Pedicularis_gracilis,Pedicularis_gracilituba,Pedicularis_gruina,Pedicularis_henryi,Pedicularis_hongii,Pedicularis_humilis,Pedicularis_inaequilobata,Pedicularis_ingens,Pedicularis_integrifolia,Pedicularis_kansuensis,Pedicularis_kariensis,Pedicularis_labordei,Pedicularis_lachnoglossa,Pedicularis_laiguensis,Pedicularis_lanpingensis,Pedicularis_lasiophrys,Pedicularis_latituba,Pedicularis_leptosiphon,Pedicularis_likiangensis,Pedicularis_limprichtiana,Pedicularis_lineata,Pedicularis_longicalyx,Pedicularis_longicaulis,Pedicularis_longiflora,Pedicularis_longipetiolata,Pedicularis_lophotrica,Pedicularis_lophotricha,Pedicularis_lutescens,Pedicularis_lyrata,Pedicularis_macilenta,Pedicularis_macrosiphon,Pedicularis_metaszetschuanica,Pedicularis_micrantha,Pedicularis_milliana,Pedicularis_minima,Pedicularis_mollis,Pedicularis_monbeigiana,Pedicularis_mussotii,Pedicularis_mychophila,Pedicularis_neolatituba,Pedicularis_nigra,Pedicularis_obliquigaleata,Pedicularis_oederi,Pedicularis_oligantha,Pedicularis_oliveriana,Pedicularis_oxycarpa,Pedicularis_paxiana,Pedicularis_pectinatiformis,Pedicularis_petitmenginii,Pedicularis_pheulpinii,Pedicularis_pinetorum,Pedicularis_polyodonta,Pedicularis_praeruptorum,Pedicularis_prezewalskii,Pedicularis_przewalskii,Pedicularis_pseudocephalantha,Pedicularis_pseudoingens,Pedicularis_pseudomelampyriflora,Pedicularis_ramosissima,Pedicularis_rex,Pedicularis_rhinanthoides,Pedicularis_rhizomatosa,Pedicularis_rhodotricha,Pedicularis_rhychodonta,Pedicularis_rhynchodonta,Pedicularis_rhynchotricha,Pedicularis_rosea,Pedicularis_rouergaiensis,Pedicularis_roylei,Pedicularis_rudis,Pedicularis_rupica,Pedicularis_rupicola,Pedicularis_scolopax,Pedicularis_semitorta,Pedicularis_sigmoidea,Pedicularis_siphonantha,Pedicularis_souliei,Pedicularis_sp.2_(jiaozishansis),Pedicularis_sphaerantha,Pedicularis_spicata,Pedicularis_stadlmanniana,Pedicularis_steiningeri,Pedicularis_stenocorys,Pedicularis_strobilacea,Pedicularis_subulatidens,Pedicularis_superba,Pedicularis_szetschuanica,Pedicularis_tachanensis,Pedicularis_tahaiensis,Pedicularis_tatsienensis,Pedicularis_tayloriana,Pedicularis_tenuisecta,Pedicularis_tenuituba,Pedicularis_thamnophila,Pedicularis_tibetica,Pedicularis_tomentosa,Pedicularis_tongolensis,Pedicularis_torta,Pedicularis_trichoglossa,Pedicularis_tricolor,Pedicularis_tristis,Pedicularis_tsiangii,Pedicularis_urceolata,Pedicularis_variegata,Pedicularis_veronicifolia,Pedicularis_verticillata,Pedicularis_wanghongiae,Pedicularis_wilsonii,Pedicularis_yanyuanensis,Pedicularis_yunnanensis,Pedicularisobliquigaleata,Latitude,Longitude
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0

# IGNORE

In [339]:
full_gdf.to_csv("/tmp/rpt.txt")

In [314]:
tmp_df = pd.read_csv("../example_data/eaton_lab_fieldnotes.csv")

# Drop one sample with ambiguous species id
#tmp_df[tmp_df["species_epithet"] == "armata/tricolor"]
tmp_df.head(20)

,expedition,accession,redo,extracted,3RAD,family,genus,species_epithet,locality,date,latitude,longitude,elevation,county,location,locality_habitat,plant_description,RNA,historical,Photo notes,HUH,NYBG,KIB,Unnamed: 23,Unnamed: 24
0,"2018 Eaton, McKenzie, Meek, Zuo",DE1,NaN,NaN,NaN,Orobanchaceae,Pedicularis,rex,1,7/7/2018,"27˚21'35"" N","99°55'01"" E",2840,Shangri-La,On highway from Shangri-la to Lijiang.,Disturbed roadside.,Burnt specimen,NaN,NaN,NaN,1,1,1.0,NaN,NaN
1,"2018 Eaton, McKenzie, Meek, Zuo",DE2,NaN,NaN,NaN,Orobanchaceae,Pedicularis,densispica,1,7/7/2018,"27˚21'35"" N","99°55'01"" E",2840,Shangri-La,On highway from Shangri-la to Lijiang.,Disturbed roadside.,Burnt specimen,NaN,NaN,NaN,1,1,1.0,NaN,NaN
2,"2018 Eaton, McKenzie, Meek, Zuo",DE3,NaN,1,x,Orobanchaceae,Pedicularis,oxycarpa,1,7/7/2018,"27˚21'35"" N","99°55'01"" E",2840,Shangri-La,On highway from Shangri-la to Lijiang.,Disturbed roadside.,Burnt specimen,NaN,NaN,NaN,1,1,1.0,NaN,NaN
3,"2018 Eaton, McKenzie, Meek, Zuo",DE4,NaN,2,NaN,Orobanchaceae,Pedicularis,longiflora,2,7/8/2018,"27°57'33.1"" N","99°42'25.8"" E",3404,Shangri-La,On highway from Shangri-la to Lijiang.,"Wet meadow w/ Primula, Juniperus.",Burnt specimen,NaN,NaN,NaN,1,1,1.0,NaN,NaN
4,"2018 Eaton, McKenzie, Meek, Zuo",DE5,NaN,2,NaN,Orobanchaceae,Pedicularis,rhinanthoides,2,7/8/2018,"27°57'33.1"" N","99°42'25.8"" E",3404,Shangri-La,On highway from Shangri-la to Lijiang.,"Wet meadow w/ Primula, Juniperus.",Burnt specimen; collected pollen; same sample ...,NaN,NaN,NaN,1,1,1.0,NaN,NaN
5,"2018 Eaton, McKenzie, Meek, Zuo",DE6,NaN,1,NaN,Orobanchaceae,Pedicularis,siphonantha,2,7/8/2018,"27°57'33.1"" N","99°42'25.8"" E",3404,Shangri-La,On highway from Shangri-la to Lijiang.,"Wet meadow w/ Primula, Juniperus.",Burnt specimen,NaN,NaN,NaN,1,1,1.0,NaN,NaN
6,"2018 Eaton, McKenzie, Meek, Zuo",DE7,NaN,NaN,NaN,Orobanchaceae,Pedicularis,densispica,2,7/8/2018,"27°57'33.1"" N","99°42'25.8"" E",3404,Shangri-La,On highway from Shangri-la to Lijiang.,"Wet meadow w/ Primula, Juniperus.",Burnt specimen; specimen discarded,NaN,NaN,NaN,0,0,0.0,NaN,NaN
7,"2018 Eaton, McKenzie, Meek, Zuo",DE8,NaN,NaN,NaN,Fabaceae,Tibetia,sp.,2,7/8/2018,"27°57'33.1"" N","99°42'25.8"" E",3404,Shangri-La,On highway from Shangri-la to Lijiang.,"Wet meadow w/ Primula, Juniperus.",Burnt specimen; Dark purple (DE8-2) and light ...,NaN,NaN,NaN,1,1,1.0,NaN,NaN
8,"2018 Eaton, McKenzie, Meek, Zuo",DE9,NaN,NaN,NaN,Fagaceae,Quercus,sp.,2,7/8/2018,"27°57'33.1"" N","99°42'25.8"" E",3404,Shangri-La,On highway from Shangri-la to Lijiang.,"Wet meadow w/ Primula, Juniperus.",Burnt specimen; DNA and pollen collected (DE9),NaN,NaN,NaN,1,1,1.0,NaN,NaN
9,"2018 Eaton, McKenzie, Meek, Zuo",DE10,NaN,NaN,NaN,Adoxaceae,Viburnum,sp.,3,7/8/2018,"28°16'51.4"" N","99°45'37.1"" E",3281,Shangri-La,On highway from Shangri-la to Lijiang.,Mixed hardwood and shrubs near streams.,Burnt specimen,NaN,NaN,NaN,1,1,1.0,NaN,NaN
